## Parse spreadsheet with AI

In [30]:
from langchain.chat_models import init_chat_model
from langchain_community.document_loaders import PolarsDataFrameLoader
import polars as pl
from pydantic import BaseModel, Field

from mfg_capabilities.config import config


pl.Config.set_tbl_rows(20)  # to set the number of rows displayed)
# pl.Config.set_tbl_cols(10) # to set the number of columns
# pl.Config.set_fmt_str_lengths(50) # to set the max string length displayed

polars.config.Config

In [31]:
COB_CAPACITY_PATH = config.data_dir / "from_central_planning" / "2025 Capacity COB.xlsx"

# Read all columns as strings first to handle concatenated CSVs with extra headers
cob_capacity_raw = pl.read_excel(
    COB_CAPACITY_PATH,
    sheet_name="KDT Q1 25 ",
    has_header=True,
    infer_schema_length=0  # Treat all columns as strings
)

cob_capacity = (
    cob_capacity_raw
    .fill_null("null")
)

display(cob_capacity)

Plant Standards Report,__UNNAMED__1,__UNNAMED__2,Line Performance By Product,__UNNAMED__4,__UNNAMED__5,__UNNAMED__6,__UNNAMED__7,__UNNAMED__8,__UNNAMED__9,__UNNAMED__10,__UNNAMED__11,__UNNAMED__12,__UNNAMED__13,__UNNAMED__14,__UNNAMED__15,__UNNAMED__16,__UNNAMED__17,__UNNAMED__18,__UNNAMED__19,__UNNAMED__20,__UNNAMED__21,__UNNAMED__22,__UNNAMED__23,__UNNAMED__24,__UNNAMED__25,__UNNAMED__26,__UNNAMED__27,__UNNAMED__28,__UNNAMED__29,__UNNAMED__30,__UNNAMED__31,__UNNAMED__32,__UNNAMED__33,__UNNAMED__34,__UNNAMED__37,__UNNAMED__38,__UNNAMED__39,__UNNAMED__40,__UNNAMED__41,__UNNAMED__42,__UNNAMED__43,__UNNAMED__44,__UNNAMED__45,__UNNAMED__46,__UNNAMED__47,__UNNAMED__48,__UNNAMED__49
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""Plant: ""","""COB""","""null""","""Date Range:""","""Q1""","""null""","""Period Range:""","""Pd 1-3 2025""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null"""
"""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""PR Calcs""","""null""","""null""","""null""","""null""","""Eff Calcs""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null"""
"""Line Name""","""Product""","""Production Units""","""Production Count""","""Uptime""","""Unplanned Downtime""","""Planned Downtime""","""Actual Rate""","""Standard Rate""","""PR""","""Eff""","""Calculated Standard Rate""","""PR @ Actual Rate""","""PR Diff Current vs Actual Rate""","""Eff @ Actual Rate""","""Eff Diff Current vs Actual Rat…","""Prop Std Rate""","""PR @ Prop Std Rate""","""PR Diff Current vs Prop Rate""","""Eff @ Prop Std Rate""","""Eff Diff Current vs Prop Rate""","""Control Point""","""Contol Point Mfg Recommended S…","""Constraint""","""Constraint Mfg Recommended Spe…","""Std Prod @ Actual Rate""","""Std Prod @ Prop Std""","""Weighted Avg Curr Std""","""Weighted Avg Prop Std""","""Weighted Avg @ Actual Rate""","""Std Prod @ Actual Rate""","""Std Prod @ Prop Std""","""Weighted Avg Curr Std""","""Weighted Avg Prop Std""","""Weighted Avg @ Actual Rate""","""Product""","""Production""","""Uptime""","""UnplannedDT""","""PlannedDT""","""Actual Rate""","""Standard Rate""","""PR""","""Eff""","""Rate Variance""","""Expected Product Count""","""Yield Losses""","""Lost Minutes"""
"""Bread Line""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""Bread Line""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null""","""null"""
"""null""","""B19011 ITALIAN BREAD""","""null""","""48385""","""10911.000122""","""2193.816556""","""627.699993""","""4.434515577""","""5""","""0.704677826""","""0.73843078""","""5""","""0.794537548""","""0.089859721""","""0.83259464""","""0.09416386""","""5""","""0.704677826""","""0""","""0.73843078""","""0""","""null""","""null""","""null""","""null""","""60897.059086875""","""68662

In [32]:
loader = PolarsDataFrameLoader(
    cob_capacity,
    page_content_column="Plant Standards Report"
)
cob_capacity_docs = loader.load()
display(cob_capacity_docs)

[Document(metadata={'__UNNAMED__1': 'COB', '__UNNAMED__2': 'null', 'Line Performance By Product ': 'Date Range:', '__UNNAMED__4': 'Q1', '__UNNAMED__5': 'null', '__UNNAMED__6': 'Period Range:', '__UNNAMED__7': 'Pd 1-3 2025', '__UNNAMED__8': 'null', '__UNNAMED__9': 'null', '__UNNAMED__10': 'null', '__UNNAMED__11': 'null', '__UNNAMED__12': 'null', '__UNNAMED__13': 'null', '__UNNAMED__14': 'null', '__UNNAMED__15': 'null', '__UNNAMED__16': 'null', '__UNNAMED__17': 'null', '__UNNAMED__18': 'null', '__UNNAMED__19': 'null', '__UNNAMED__20': 'null', '__UNNAMED__21': 'null', '__UNNAMED__22': 'null', '__UNNAMED__23': 'null', '__UNNAMED__24': 'null', '__UNNAMED__25': 'null', '__UNNAMED__26': 'null', '__UNNAMED__27': 'null', '__UNNAMED__28': 'null', '__UNNAMED__29': 'null', '__UNNAMED__30': 'null', '__UNNAMED__31': 'null', '__UNNAMED__32': 'null', '__UNNAMED__33': 'null', '__UNNAMED__34': 'null', '__UNNAMED__37': 'null', '__UNNAMED__38': 'null', '__UNNAMED__39': 'null', '__UNNAMED__40': 'null', '__

In [37]:
class Products(BaseModel):
    """
    Represents the product data from the production sheet.
    """
    products: list["Product"] = Field(
        "List of Product objects representing production data. Each record corresponds to a line"
        "and product combination. It does not include subtotals and other summary rows."
    )


class Product(BaseModel):
    """
    Represents a single product from the production data.
    """
    plant: str = Field("Plant from the the top of the sheet.")
    date_range: str = Field("Date Range from the top of the sheet, e.g. Q1")
    period_range: str = Field("Period Range from the top of the sheet, e.g. pd 1-3 2025")
    line: str = Field("Line Name column. The line within the plant where the production occurs.")
    product: str = Field("Product Column. The product being produced on the line.")
    production_count: int = Field(
        "Production Count column. The number of units produced." \
        "Rounded to the nearest integer if necessary."
    )


llm = init_chat_model(model="gpt-4o", model_provider="openai")
structured_llm = llm.with_structured_output(Products)


prompt_template = """
You will be given a list of rows parsed from an Excel sheet. The sheet had summary
fields at the top and other formatting quirks. You should extract the requested data
data such that we can turn it into a clean table of lines, products, and production
counts. Do not include any rows that contain a subtotal or null or zero-valued product
column. The raw data follows: {raw_data}
""" 

num_docs = len(cob_capacity_docs)
batch_size = 20
all_products = []
initial_docs = cob_capacity_docs[:4] # the header rows

# Could turn this into a batch call with rate limiting
for i in range(4, num_docs, batch_size):
    current_batch = cob_capacity_docs[i:i+batch_size]
    batch_for_prompt = initial_docs + current_batch
    formatted_prompt = prompt_template.format(raw_data=batch_for_prompt)
    response = structured_llm.invoke(formatted_prompt)
    
    if response.products:
        all_products.extend(response.products)

In [38]:
response_df = pl.DataFrame(all_products)
display(response_df)

plant,date_range,period_range,line,product,production_count
str,str,str,str,str,i64
"""COB""","""Q1""","""Pd 1-3 2025""","""Bread Line""","""B19011 ITALIAN BREAD""",48385
"""COB""","""Q1""","""Pd 1-3 2025""","""Bread Line""","""B19017 COUNTRY FRENCH""",25744
"""COB""","""Q1""","""Pd 1-3 2025""","""Bread Line""","""B19019 SEEDED RYE BREAD""",3356
"""COB""","""Q1""","""Pd 1-3 2025""","""Bread Line""","""B19068 PUMPERNICKEL BREAD""",2944
"""COB""","""Q1""","""Pd 1-3 2025""","""Bread Line""","""B19100 FRENCH STIX""",1681
"""COB""","""Q1""","""Pd 1-3 2025""","""Bread Line""","""B19180 SWEET ITALIAN STIX""",6396
"""COB""","""Q1""","""Pd 1-3 2025""","""Bread Line""","""B19183 WHITE MOUNTAIN BREAD""",872
"""COB""","""Q1""","""Pd 1-3 2025""","""Bread Line""","""B19184 Wheat Mountain""",345
"""COB""","""Q1""","""Pd 1-3 2025""","""Bread Line""","""B19185 ROUND PUMPERNICKLE""",752
